In [1]:
import os
import wrf
import matplotlib
import numpy as np
import xarray as xr
import rioxarray as rio
import pandas as pd
import netCDF4 as nc
import cartopy.crs as crs
import matplotlib.pyplot as plt
import matplotlib.image as image
import matplotlib.colors as colors
from metpy.plots import USCOUNTIES
from cartopy.feature import NaturalEarthFeature
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim, cartopy_ylim, latlon_coords)
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import rcParams
import matplotlib.colors as mcolors
from pathlib import Path
rcParams['font.family']     = 'sans-serif'
rcParams['font.sans-serif'] = 'Helvetica'
plt.rcParams.update({'figure.autolayout': True})



In [2]:
#-- Define filepaths
root_dir = Path().resolve().parents[1]
data_dir = root_dir / 'data' / '01-raw' / 'geospatial' / 'goes-west-lcl'
fname = 'cldalb-sci.nc'
fpath = str(data_dir / fname)

out_dir = root_dir / 'outputs' / 'clc-plots' / 'albedo-plots'
#out_dir = '/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis/outputs/spatial-analysis-figs/clc-binary'

print('----------------------------')
print("Root Directory: ", root_dir)
print("Data Directory: ", data_dir)
print("Out Directory: ", out_dir)
print('----------------------------')

----------------------------
Root Directory:  /Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-fog-analysis
Data Directory:  /Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-fog-analysis/data/01-raw/geospatial/goes-west-lcl
Out Directory:  /Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-fog-analysis/outputs/clc-plots/albedo-plots
----------------------------


# Make terrain height dataset

In [11]:
# Prepare terrain data
dem_fpath = '/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-fog-analysis/data/01-raw/geospatial/sci-dem/sci/sci-dem-500m.tif'
dem_da = rio.open_rasterio(dem_fpath)
dem_da = dem_da.rename({"x" : "lon", "y" : "lat"})
dem_da = dem_da.squeeze()
dem_da = xr.where(dem_da < 0, 0, dem_da) # Set everything below sea level to 0

# Make Binary LCL Plots

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors

# Load the dataset
ds = xr.open_dataset(fpath)
cldalb = ds['cldalb']

# Make binary LCL 
cldalb_binary = xr.where(cldalb > 8.5, 1, 0)

# Santa Cruz Island boundaries
lonmin, lonmax = -119.98, -119.48
latmin, latmax = 33.80, 34.20

# Define station locations
sauclat = 34.001033
sauclon = -119.817817

upemlat = 34.012531
upemlon = -119.801828


min_time = '2008-08-16 17:00:00'
max_time = '2008-08-18 16:00:00'
cldalb_binary = cldalb_binary.sel(time=slice(min_time, max_time))

# Iterate over each timestamp in the 'time' dimension
for i, timestamp in enumerate(cldalb_binary.time):
    timestamp_dt = pd.to_datetime(timestamp.values).round('30min')
    minutes = timestamp_dt.minute
    if minutes == 0:
        # Extract 'cldalb' data for this timestamp
        cldalb_at_time = cldalb_binary.isel(time=i).sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))

        # Create figure and axis with Cartopy projection
        fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})

        # Set map extent
        ax.set_extent([lonmin, lonmax, latmin, latmax], crs=ccrs.PlateCarree())

        # Add land, ocean, and borders
        ax.add_feature(cfeature.OCEAN, facecolor="blue", alpha=1, zorder=10)
        ax.coastlines()

        # Create a mesh grid for terrain contours
        lon2d, lat2d = np.meshgrid(dem_da.lon, dem_da.lat)

        # Custom colormap: Truncate "terrain" colormap to exclude blue
        terrain_cmap = plt.get_cmap("terrain")
        custom_cmap = mcolors.LinearSegmentedColormap.from_list(
            "custom_terrain", terrain_cmap(np.linspace(0.25, 1.0, 256))
        )

        # Plot terrain contours
        terrain_contour = ax.contourf(lon2d, lat2d, dem_da.values, levels=20, 
                                    cmap=custom_cmap, transform=ccrs.PlateCarree(), zorder=1)
        
        ax.scatter([sauclon, upemlon], [sauclat, upemlat], color='red', zorder=40, s=40, label='Stations', transform=ccrs.Geodetic())
        
        # Create a custom colormap: 0 is transparent, 1 is white
        binary_cmap = mcolors.ListedColormap([(0, 0, 0, 0), (1, 1, 1, 1)])  # Transparent for 0, white for 1

        # Overlay cloud albedo as a semi-transparent layer
        # Overlay cloud albedo as a semi-transparent layer using imshow
        cldalb_plot = ax.imshow(cldalb_at_time.values, cmap=binary_cmap, origin='lower', 
                                extent=[cldalb_at_time.lon.min(), cldalb_at_time.lon.max(),
                                        cldalb_at_time.lat.min(), cldalb_at_time.lat.max()],
                                alpha=0.75, zorder=20)

        # Add a title with the timestamp
        time = pd.to_datetime(timestamp.values).ceil('30min')
        ax.set_title(f'Binary LCL Map - {time}')

        # Show the plot
        #out_dir = root_dir / 'outputs' / 'clc-plots' / 'binary-clc-plots'
        out_dir = '/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis/outputs/spatial-analysis-figs/clc-binary'
        fname = f'{str(i).zfill(2)}-binary-lcl-{time}'
        plt.savefig(os.path.join(out_dir, fname))
        plt.close()

print (out_dir + fname)

/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis/outputs/spatial-analysis-figs/clc-binary48-binary-lcl-2008-08-18 16:00:00


In [27]:
pd.to_datetime(timestamp.values).minute

0

# Make Albedo Plots

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Load dataset
ds = xr.open_dataset(fpath)
cldalb = ds['cldalb']

# Santa Cruz Island boundaries
lonmin, lonmax = -119.98, -119.48
latmin, latmax = 33.80, 34.20

# Define station locations
sauclat = 34.001033
sauclon = -119.817817

upemlat = 34.012531
upemlon = -119.801828

# Define where to save
out_dir = root_dir / 'outputs' / 'clc-plots' / 'albedo-plots'

min_time = '2008-08-23 17:00:00'
max_time = '2008-08-25 16:00:00'
cldalb = cldalb.sel(time=slice(min_time, max_time))

# Define contour levels (0 to max height)
max_value = np.nanmax(dem_da.values)  # Find max height
num_levels = 10  # Adjust for smoother contours
contour_levels = np.linspace(100, max_value, num_levels)  # Levels from 0

# Iterate over timestamps in cldalb
for i, timestamp in enumerate(cldalb.time):
    timestamp_dt = pd.to_datetime(timestamp.values).round('30min')
    minutes = timestamp_dt.minute
    if minutes == 0:

        # Subset cloud albedo data
        cldalb_at_time = cldalb.isel(time=i).sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))

        # Create figure and axis with Cartopy projection
        fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})

        # Set map extent
        ax.set_extent([lonmin, lonmax, latmin, latmax], crs=ccrs.PlateCarree())

        # Add base map features
        ax.add_feature(cfeature.OCEAN, facecolor="lightgrey", alpha=1, zorder=10)
        ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgrey')  # Remove land color
        ax.add_feature(cfeature.BORDERS, linestyle=":")
        ax.coastlines()

        # Create a mesh grid for contouring
        lon2d, lat2d = np.meshgrid(dem_da.lon, dem_da.lat)

        # Plot only terrain contours (no color fill)
        terrain_contour = ax.contour(lon2d, lat2d, dem_da.values, 
                                    levels=3, colors="black", linewidths=1, transform=ccrs.PlateCarree(),
                                    zorder=1)
        
        ax.scatter([sauclon, upemlon], [sauclat, upemlat], color='red', zorder=40, s=40, label='Stations', transform=ccrs.Geodetic())

    # Normalize cloud albedo range from 0 to 80 for colormap
        norm = mcolors.Normalize(vmin=0, vmax=80)
        
        # Overlay cloud albedo as a semi-transparent layer
        cldalb_plot = ax.pcolormesh(cldalb_at_time.lon, cldalb_at_time.lat, cldalb_at_time.values, 
                                    cmap="jet", alpha=0.8, transform=ccrs.PlateCarree(), zorder=20, norm=norm)

        # Add colorbars
        cbar_cldalb = plt.colorbar(cldalb_plot, ax=ax, orientation='horizontal', pad=0.05, label='Cloud Albedo')
        cbar_cldalb.set_ticks([0, 20, 40, 60, 80])  # Optional: set specific ticks for clarity


        # Add title with timestamp
        time = pd.to_datetime(timestamp.values).ceil('30min')
        ax.set_title(f'Cloud Albedo - {time}')

        out_dir = '/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis/outputs/spatial-analysis-figs/clc-albedo'
        fname = f'{str(i).zfill(2)}-albedo-{time}.png'
        plt.savefig(os.path.join(out_dir, fname))
        plt.close()
    # Show the plot
    #plt.show()

# Monthly Mean Cloud Albedo Plots

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Load dataset
ds = xr.open_dataset(fpath)

# Average data by month
ds_months = ds.groupby('time.month').mean('time')

# Santa Cruz Island boundaries
lonmin, lonmax = -119.98, -119.48
latmin, latmax = 33.80, 34.20

# Define station locations
sauclat = 34.001033
sauclon = -119.817817

upemlat = 34.012531
upemlon = -119.801828

# Define where to save
out_dir = root_dir / 'outputs' / 'clc-plots' / 'monthly-average-albedo'

# Define contour levels (0 to max height)
max_value = np.nanmax(dem_da.values)  # Find max height
num_levels = 10  # Adjust for smoother contours
contour_levels = np.linspace(100, max_value, num_levels)  # Levels from 0

# Iterate over timestamps in cldalb
for i in ds_months.month.values:
    # Subset cloud albedo data
    cldalb = ds_months.sel(month=i).sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))
    cldalb = cldalb['cldalb']
    
    # Create figure and axis with Cartopy projection
    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})

    # Set map extent
    ax.set_extent([lonmin, lonmax, latmin, latmax], crs=ccrs.PlateCarree())

    # Add base map features
    ax.add_feature(cfeature.OCEAN, facecolor="lightgrey", alpha=1, zorder=10)
    ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgrey')  # Remove land color
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.coastlines()

    # Create a mesh grid for contouring
    lon2d, lat2d = np.meshgrid(dem_da.lon, dem_da.lat)

    # Plot only terrain contours (no color fill)
    terrain_contour = ax.contour(lon2d, lat2d, dem_da.values, 
                                 levels=3, colors="black", linewidths=1, transform=ccrs.PlateCarree(),
                                 zorder=1)
    
    ax.scatter([sauclon, upemlon], [sauclat, upemlat], color='red', zorder=40, s=40, label='Stations', transform=ccrs.Geodetic())

   # Normalize cloud albedo range from 0 to 80 for colormap
    norm = mcolors.Normalize(vmin=0, vmax=25)
    
    # Overlay cloud albedo as a semi-transparent layer
    cldalb_plot = ax.pcolormesh(cldalb.lon, cldalb.lat, cldalb.values, 
                                cmap="jet", alpha=0.8, transform=ccrs.PlateCarree(), zorder=20, norm=norm)

    # Add colorbars
    cbar_cldalb = plt.colorbar(cldalb_plot, ax=ax, orientation='horizontal', pad=0.05, label='Cloud Albedo')
    cbar_cldalb.set_ticks([0, 5, 10, 15, 20, 25])  # Optional: set specific ticks for clarity

    ax.set_title(f'Cloud Albedo Average - Month {i}')

    fname = f'{i}-average-albedo.png'
    plt.savefig(os.path.join(out_dir, fname))
    plt.close()
    # Show the plot
    #plt.show()

# Cloud Count per Month

In [15]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Load dataset
ds = xr.open_dataset(fpath)
ds['cldalb'] = xr.where(ds['cldalb'] > 8.5, 1, 0)  # Replace with a binary LCL / Clear

# Get percentage of observations that are cloudy, broken down by month
total_count = ds['time'].shape[0]
ds_months = ds.groupby('time.month').sum('time') / total_count * 100


# Santa Cruz Island boundaries
lonmin, lonmax = -119.98, -119.48
latmin, latmax = 33.80, 34.20

# Define station locations
sauclat = 34.001033
sauclon = -119.817817

upemlat = 34.012531
upemlon = -119.801828

# Define where to save
out_dir = root_dir / 'outputs' / 'clc-plots' / 'cloud-frequency-monthly'

# Define contour levels (0 to max height)
max_value = np.nanmax(dem_da.values)  # Find max height
num_levels = 10  # Adjust for smoother contours
contour_levels = np.linspace(100, max_value, num_levels)  # Levels from 0

# Iterate over timestamps in cldalb
for i in ds_months.month.values:
    # Subset cloud albedo data
    cldalb = ds_months.sel(month=i).sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))
    cldalb = cldalb['cldalb']
    
    # Create figure and axis with Cartopy projection
    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})

    # Set map extent
    ax.set_extent([lonmin, lonmax, latmin, latmax], crs=ccrs.PlateCarree())

    # Add base map features
    ax.add_feature(cfeature.OCEAN, facecolor="lightgrey", alpha=1, zorder=10)
    ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgrey')  # Remove land color
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.coastlines()

    # Create a mesh grid for contouring
    lon2d, lat2d = np.meshgrid(dem_da.lon, dem_da.lat)

    # Plot only terrain contours (no color fill)
    terrain_contour = ax.contour(lon2d, lat2d, dem_da.values, 
                                 levels=3, colors="black", linewidths=1, transform=ccrs.PlateCarree(),
                                 zorder=1)
    
    ax.scatter([sauclon, upemlon], [sauclat, upemlat], color='red', zorder=40, s=40, label='Stations', transform=ccrs.Geodetic())

   # Normalize cloud albedo range from 0 to 80 for colormap
    norm = mcolors.Normalize(vmin=0, vmax=15)
    
    # Overlay cloud albedo as a semi-transparent layer
    cldalb_plot = ax.pcolormesh(cldalb.lon, cldalb.lat, cldalb.values, 
                                cmap="jet", alpha=0.8, transform=ccrs.PlateCarree(), zorder=20, norm=norm)

    # Add colorbars
    cbar_cldalb = plt.colorbar(cldalb_plot, ax=ax, orientation='horizontal', pad=0.05, label='Percentage of Observations where Albedo > 8.5%')
    #cbar_cldalb.set_ticks([0, 5, 10, 15, 20, 25])  # Optional: set specific ticks for clarity

    ax.set_title(f'GOES Low Cloud Frequency Aggregated by Month\nMonth {i}\nDaylight Hours')

    fname = f'{i}-month-cloud-frequency.png'
    plt.savefig(os.path.join(out_dir, fname))
    plt.close()
    # Show the plot
    #plt.show()

# Hourly Mean Albedo Plots

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Load dataset
ds = xr.open_dataset(fpath)

# Average data by hour
ds_hours = ds.groupby('time.hour').mean('time')

# Santa Cruz Island boundaries
lonmin, lonmax = -119.98, -119.48
latmin, latmax = 33.80, 34.20

# Define station locations
sauclat = 34.001033
sauclon = -119.817817

upemlat = 34.012531
upemlon = -119.801828

# Define where to save
out_dir = root_dir / 'outputs' / 'lcl-plots' / 'hourly-average-albedo'

# Define contour levels (0 to max height)
max_value = np.nanmax(dem_da.values)  # Find max height
num_levels = 10  # Adjust for smoother contours
contour_levels = np.linspace(100, max_value, num_levels)  # Levels from 0

# Iterate over timestamps in cldalb
for i in ds_hours.hour.values:
    # Subset cloud albedo data
    cldalb = ds_hours.sel(hour=i).sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))
    cldalb = cldalb['cldalb']
    
    # Create figure and axis with Cartopy projection
    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})

    # Set map extent
    ax.set_extent([lonmin, lonmax, latmin, latmax], crs=ccrs.PlateCarree())

    # Add base map features
    ax.add_feature(cfeature.OCEAN, facecolor="lightgrey", alpha=1, zorder=10)
    ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgrey')  # Remove land color
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.coastlines()

    # Create a mesh grid for contouring
    lon2d, lat2d = np.meshgrid(dem_da.lon, dem_da.lat)

    # Plot only terrain contours (no color fill)
    terrain_contour = ax.contour(lon2d, lat2d, dem_da.values, 
                                 levels=3, colors="black", linewidths=1, transform=ccrs.PlateCarree(),
                                 zorder=1)
    
    ax.scatter([sauclon, upemlon], [sauclat, upemlat], color='red', zorder=40, s=40, label='Stations', transform=ccrs.Geodetic())

   # Normalize cloud albedo range from 0 to 80 for colormap
    norm = mcolors.Normalize(vmin=0, vmax=35)
    
    # Overlay cloud albedo as a semi-transparent layer
    cldalb_plot = ax.pcolormesh(cldalb.lon, cldalb.lat, cldalb.values, 
                                cmap="jet", alpha=0.8, transform=ccrs.PlateCarree(), zorder=20, norm=norm)

    # Add colorbars
    cbar_cldalb = plt.colorbar(cldalb_plot, ax=ax, orientation='horizontal', pad=0.05, label='Cloud Albedo')
    cbar_cldalb.set_ticks([0, 5, 10, 15, 20, 25, 30, 35])  # Optional: set specific ticks for clarity

    ax.set_title(f'Cloud Albedo Average - Hour {i}')

    fname = f'{i}-average-albedo.png'
    plt.savefig(os.path.join(out_dir, fname))
    plt.close()
    # Show the plot
    #plt.show()

# Cloud Count per Hour

In [21]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Load dataset
ds = xr.open_dataset(fpath)
ds['cldalb'] = xr.where(ds['cldalb'] > 8.5, 1, 0)  # Replace with a binary LCL / Clear

# get percentage of cloudy observations, by hour
total_count = ds['time'].shape[0]
ds_hours = ds.groupby('time.hour').sum('time') / total_count * 100

# Santa Cruz Island boundaries
lonmin, lonmax = -119.98, -119.48
latmin, latmax = 33.80, 34.20

# Define station locations
sauclat = 34.001033
sauclon = -119.817817

upemlat = 34.012531
upemlon = -119.801828

# Define where to save
out_dir = root_dir / 'outputs' / 'clc-plots' / 'cloud-frequency-hourly'

# Define contour levels (0 to max height)
max_value = np.nanmax(dem_da.values)  # Find max height
num_levels = 10  # Adjust for smoother contours
contour_levels = np.linspace(100, max_value, num_levels)  # Levels from 0

# Iterate over timestamps in cldalb
for i in ds_hours.hour.values:
    # Subset cloud albedo data
    cldalb = ds_hours.sel(hour=i).sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))
    cldalb = cldalb['cldalb']
    
    # Create figure and axis with Cartopy projection
    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})

    # Set map extent
    ax.set_extent([lonmin, lonmax, latmin, latmax], crs=ccrs.PlateCarree())

    # Add base map features
    ax.add_feature(cfeature.OCEAN, facecolor="lightgrey", alpha=1, zorder=10)
    ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgrey')  # Remove land color
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.coastlines()

    # Create a mesh grid for contouring
    lon2d, lat2d = np.meshgrid(dem_da.lon, dem_da.lat)

    # Plot only terrain contours (no color fill)
    terrain_contour = ax.contour(lon2d, lat2d, dem_da.values, 
                                 levels=3, colors="black", linewidths=1, transform=ccrs.PlateCarree(),
                                 zorder=1)
    
    ax.scatter([sauclon, upemlon], [sauclat, upemlat], color='red', zorder=40, s=40, label='Stations', transform=ccrs.Geodetic())

    # Normalize cloud albedo range from 0 to 80 for colormap
    norm = mcolors.Normalize(vmin=0, vmax=6)
    
    # Overlay cloud albedo as a semi-transparent layer
    cldalb_plot = ax.pcolormesh(cldalb.lon, cldalb.lat, cldalb.values, 
                                cmap="jet", alpha=0.8, transform=ccrs.PlateCarree(), zorder=20, norm=norm)

    # Add colorbars
    cbar_cldalb = plt.colorbar(cldalb_plot, ax=ax, orientation='horizontal', pad=0.05, label='Percentage of Observations where Albedo > 8.5%')
    #cbar_cldalb.set_ticks([0, 5, 10, 15, 20, 25, 30, 35])  # Optional: set specific ticks for clarity

    ax.set_title(f'GOES Low Cloud Frequency Aggregated by Hour\nHour {i}\nSummer Months')

    fname = f'{i}-hourly-cloud-frequency.png'
    plt.savefig(os.path.join(out_dir, fname))
    plt.close()
    # Show the plot
    #plt.show()